In [1]:
import requests
import json
import tqdm
import datetime
from bs4 import BeautifulSoup as BS

In [2]:
def get_text_datetime(url) :
    r = requests.get(url)
    try :
        bs = BS(r.text)
        text = bs.find(id = 'newsct_article').text
        datetime = bs.find('span', {'class' : 'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'})['data-date-time']
        return {'text' : clean(text), 'datetime' : datetime}
    except :
        return {'error' : 'error'}
        
def clean(text) :
    return text.replace('\n', '').strip()

def get_sids() :
    row = {
        '정치' : 100,
        '경제' : 101,
        '사회' : 102,
        '생활/문화' : 103,
        '세계' : 104,
        'IT/과학' : 105,
    }
    return row

In [ ]:
main_url = 'https://news.naver.com/main/mainNews.naver'
target_day = -1 # 어제
target_datetime = datetime.datetime.today().date() + datetime.timedelta(days = target_day)
sids = get_sids()
payload = {
    'sid1' : None,
    'date' : '00:00:00',
    'page' : 1,   
}
results = [] # 기사들을 담을 리스트
test = 1
for sid in tqdm.tqdm(sids.keys()) :
    payload['sid1'] = sids[sid]
    page = 0
    stop = False
    while not stop :
        page += 1
        payload['page'] = page
        r = requests.post(main_url, data = payload)
        data = r.json()
        for news in json.loads(data['airsResult'])['result'][f"{sids[sid]}"] :
            title = news['title']
            url = f"https://n.news.naver.com/mnews/article/{news['officeId']}/{news['articleId']}"
            text_datetime = get_text_datetime(url)
            row = {
                'type' : sid,
                'title' : title,
                'url' : url,
                **text_datetime
            }
            try : # 정상적인 경우
                if datetime.datetime.strptime(row['datetime'], '%Y-%m-%d %H:%M:%S').date() > target_datetime : # 대상 날짜에 이르기 전
                    continue
                elif datetime.datetime.strptime(row['datetime'], '%Y-%m-%d %H:%M:%S').date() == target_datetime : # 대상 날짜
                    results.append(row)
                else : # 대상 날짜를 지난 경우 
                    stop = True
                    break
                     
            except : # error 발생한 경우
                results.append(row) # error에 경우 추후 처리

 83%|███████████████████████████████████████████████████████████████████▌             | 5/6 [1:23:41<14:14, 854.27s/it]

In [ ]:
file_name = f"{target_datetime.year}{str(target_datetime.month).zfill(2)}{str(target_datetime.day).zfill(2)}_news.json"
text = json.dumps(results, ensure_ascii=False)
try :
    with open(f"C:/news/{file_name}", 'w', encoding = 'utf-8') as file :
        file.write(text)
except :
    import os
    os.mkdir('C:/news')
    with open(f"C:/news/{file_name}", 'w', encoding = 'utf-8') as file :
        file.write(text)